## Werewolf Game Video Analysis Project
#### Experimental design and implementation by K. Sozykin
#### Dataset, idea by Nurmukhamet Abdullin

In [ ]:
%pylab inline

In [ ]:
import scipy as sp
import pandas as pd
import numpy as np
np.random.seed(42)
import seaborn as sns
import cv2
import os

In [ ]:
import datetime

In [ ]:
import tensorflow as tf
tf.set_random_seed(42)

In [ ]:
from scipy.signal import medfilt
from scipy.signal import butter,filtfilt,firwin,medfilt,lfilter

In [ ]:
from tqdm import tqdm

In [ ]:
from keras.applications.vgg19 import VGG19

from keras.models import Model
from keras.optimizers import SGD,Adam
from keras.layers.convolutional import Convolution1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import Sequential

from keras.layers import Dense,Dropout, Activation,LSTM,Flatten,Bidirectional
from keras.wrappers.scikit_learn import KerasClassifier

from keras_squeezenet import SqueezeNet

In [ ]:
from sklearn.metrics import accuracy_score , f1_score,jaccard_similarity_score,confusion_matrix
from sklearn.metrics import zero_one_loss
from sklearn.metrics import hamming_loss
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.utils import shuffle as sk_shuffle

In [ ]:
import xgboost as xgb

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from  xgboost import XGBClassifier

In [ ]:
import librosa

In [ ]:
import pickle

In [ ]:
import hickle as hkl

In [ ]:
from keras.utils import plot_model

## Constants

In [ ]:
global wsize
wsize = 25

In [ ]:
vids_dir = os.path.join('..','data','video')
auds_dir = os.path.join('..','data','audio')
label_path = os.path.join('..','data','ground_truth2.xlsx')

In [ ]:
assert os.path.exists(vids_dir) and os.path.exists(auds_dir) and os.path.exists(label_path)

## Utility functions

In [ ]:
from utils import  *

In [ ]:
def vid2data_cv2(vname,vid_path,interv = None,grayscale = False,res_c = 1,starts = []):
    """
    
    """
    try:
        cap = cv2.VideoCapture(vid_path)
    except:
        print ("problem opening input stream")
        return
    if not cap.isOpened():
        print ("capture stream not open")
        return
    prev = 0
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps    = int(cap.get(cv2.CAP_PROP_FPS))
    global wsize
    wsize = int(fps)
    print("%s has fps %s len %s" %(vname,fps,length))
    dataset = {
        'vid_features' : [[] for e in range(13)],
        'motion_series' : [[] for e in range(13)],
        'labels' : [],
        vname : [[] for e in range(13)],
    }
    starts = fps * array(starts)
    step = 1
    ok_idx = ones([length,]).astype(bool)
    sample_windows = [0] +  np.random.permutation(arange(1,11))[:3].tolist()
    print('sample windows',sample_windows)
    ok_idx[:starts[0]] = False 
    corr_idx = array([1,2,3,4,5,6,9,10,7,8])-1
    for fr_idx in tqdm(range(length)):
        
        ret, fr = cap.read()
        
        if (fr_idx > 0 and fr_idx % fps == 0):

            
            q1 = qsplit(fr)
            q2 = [qsplit(e) for e in q1[1:]]
            player_shape = q2[0][0].shape
            main_window = [q1[0]]
            players_windows = [e for e in array(q2).reshape([12] + list(player_shape))]
            players_windows = [players_windows[idx] for idx in corr_idx]
            
            windows = main_window + players_windows
            
            diff = np.abs(array(prev)-array(players_windows))
            for j,e in enumerate(dataset['motion_series']):
                dataset['motion_series'][j].append(sum(sum(sum(diff))))
            prev = players_windows
            # vgg 224x224
            target_size = (227, 227)
            windows = array([ cv2.resize(e,target_size).astype('float32') for e in windows ])
            
            #windows = windows[sample_windows]
            features  = imnet_features(windows)
            for idx1,idx2 in enumerate(sample_windows):
                dataset['vid_features'][idx1].append(features[idx2])
            for idx,f in enumerate(features):
                dataset[vname][idx].append(f) 
            lb = 0
            cur_sec = int(round(fr_idx/fps))
            if interv != None:
                for k in range(0,len(interv),2):
                    if cur_sec >= interv[k] and cur_sec <= interv[k + 1]:
                        lb = 1
                        break 

            dataset['labels'].append(lb)
    cap.release()
    return dataset

In [ ]:
vid2data = vid2data_skv if os.name == 'posix' else vid2data_cv2

In [ ]:
def lowpass_filter(x,order = 10,Fc = 40, Fs = 1600):
    # provide them to firwin
    h = firwin(numtaps=order, cutoff=Fc, nyq=Fs/2)
    # 'x' is the time-series data you are filtering
    y = lfilter(h, 1.0, x)
    return y

In [ ]:
def nn_model_video(opt = Adam(),nb_lstm1 = 256,p = 0.5,data_dim = 1000,timesteps = wsize,nb_classes=2):
    # expected input data shape: (batch_size, timesteps, data_dim)
    model = Sequential()
    
    model.add(Bidirectional(LSTM(nb_lstm1, return_sequences=True,activation = 'linear'),input_shape = [timesteps,data_dim]))
    model.add(Dropout(p))
    model.add(Dense(nb_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', 
                  optimizer = opt)
    return model

In [ ]:
def nn_model_audio(opt = Adam(),nb_lstm1 = 256,p = 0.5,data_dim = 1000,timesteps = wsize,nb_classes=2):
    # expected input data shape: (batch_size, timesteps, data_dim)
    model = Sequential()
    
    model.add(Bidirectional(LSTM(nb_lstm1, return_sequences=True,activation = 'sigmoid'),input_shape = [timesteps,data_dim]))
    model.add(Dropout(p))
    model.add(Bidirectional(LSTM(nb_lstm1//2, return_sequences=True,activation = 'sigmoid')))
    model.add(Dropout(p)),
    model.add(Dense(nb_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', 
                  optimizer = opt)
    return model

## ImageNet Model

In [ ]:
#imnet = VGG19(weights='imagenet')
#imnet_extractor = Model(input=imnet.input, output=imnet.get_layer('fc1').output)
imnet = SqueezeNet()
imnet_extractor = Model(inputs=imnet.input, outputs=imnet.get_layer('loss').output)

In [ ]:
def imnet_features(x):
    """
        return vgg fc1 features for particlurar image 
    """
    mean = [103.939,116.779,123.68]
    x = array([ (e-mean) for e in x])
    return imnet_extractor.predict(x)

## data

In [ ]:
interv_cols = "Day0 Night1 Day1 Night2 Day2 Night3 Day3 Night4 Day4 Night5".split(' ')

In [ ]:
xls = pd.ExcelFile(label_path)

In [ ]:
xls

In [ ]:
gtruth = xls.parse('main')

In [ ]:
vlist = file_list(vids_dir,'Cap02*.mp4')

In [ ]:
#vlist = np.random.permutation(vlist)

In [ ]:
vlist

In [ ]:
dataset = {}

In [ ]:
dataset['vlist'] = vlist

## Feature extraction From Video

In [ ]:
for idx,v in enumerate(vlist[:]):
    sep = '/' if os.name == 'posix' else '\\' 
    vname = v.split(sep)[-1]
    match_sz = len(gtruth[gtruth.File == vname][interv_cols].values)
    #dataset['vlist'].append(vname)
    starts =  []
    ends =  []
    interv = []
    for i in range(match_sz):
        interv += [t2sec(e)  for e in gtruth[gtruth.File == vname][interv_cols].values[i] if str(e)!='nan']
    
    starts += [t2sec(e)  for e in gtruth[gtruth.File == vname]['Start'].values if str(e)!='nan']
    ends += [t2sec(e)  for e in gtruth[gtruth.File == vname]['End'].values if str(e)!='nan']
    print(v,interv)
    ds = vid2data(vname,v,interv = interv,starts = starts)
    if idx == 0:
        dataset = ds
    else:
        dataset['labels'] += ds['labels']
        dataset[vname] = ds[vname]
        for i in range(len(dataset['vid_features'])):
            dataset['motion_series'][i] += ds['motion_series'][i]
            dataset['vid_features'][i] += ds['vid_features'][i]

## Feature extraction From Audio

In [ ]:
alist = array([e.replace('video','audio').replace('mp4','opus') for e in vlist])

In [ ]:
dataset['audio_features'] = []

In [ ]:
def split_audio(fname):
    r = []
    x,fs = librosa.load(fname)
    sec_step = x.shape[0]//fs
    for t in range(0,sec_step):
        sec = x[t*fs+1:(t+1)*fs]
        r.append(sec)
    return (array(r),fs)

def get_mfcc(spec,fs):
    mfccf = array([librosa.feature.mfcc(e,fs,n_mfcc = 40) for e in spec])
    mfccf = array([e.flatten() for e in mfccf])
    mfccf = (mfccf - mfccf.mean(0))/mfccf.std(0)
    return mfccf

def get_chroma_stft(spec,fs):
    chroma_stft = array([librosa.feature.chroma_stft(e,fs) for e in spec])
    chroma_stft = array([e.flatten() for e in chroma_stft])
    chroma_stft = (chroma_stft - chroma_stft.mean(0))/chroma_stft.std(0)
    return chroma_stft

def get_spectral_center(spec,fs):
    spectral_center = array([librosa.feature.spectral_centroid(e,fs) for e in spec])
    spectral_center = array([e.flatten() for e in spectral_center])
    spectral_center = (spectral_center - spectral_center.mean(0))/spectral_center.std(0)
    return spectral_center

def get_spectral_rolloff(spec,fs):
    spectral_rolloff = array([librosa.feature.spectral_rolloff(e,fs) for e in spec])
    spectral_rolloff = array([e.ravel() for e in spectral_rolloff])
    spectral_rolloff = (spectral_rolloff - spectral_rolloff.mean(0))/spectral_rolloff.std(0)
    return spectral_rolloff

In [ ]:
for e in alist[:]:
    
    aname = e.split('\\')[-1]
    print(e,aname)
    wave,fs = split_audio(e)
    mfccf = get_mfcc(wave,fs)
    spectral_center = get_spectral_center(wave,fs)
    chroma_stft = get_chroma_stft(wave,fs)
    spectral_rolloff = get_spectral_rolloff(wave,fs)
    t = hstack([mfccf,spectral_center,spectral_rolloff])
    dataset[aname] =  t
    dataset['audio_features'] += t.tolist()

## Load pre-extracted Data

In [ ]:
do_load = False

In [ ]:
if do_load:
    fname = 'dataset_2017-04-25__02-41-36.pkl'
    with open(fname,'rb') as f:
        dataset = pickle.load(f)

## Models Training

In [ ]:
assert

In [ ]:
def day_night_experiment(experiment_type = 2,verbose = 0):
    U,V = [],[]
    # number of player frames
    N = 3
    if experiment_type == 1:
        nb_epoch=100
        avg_motion = np.mean(dataset['motion_series'],0)
        avg_motion = lowpass_filter(avg_motion)
        avg_motion = (avg_motion -  avg_motion.mean())/avg_motion.std()
        motion_lbs = array(dataset['labels'])
        motion_lbs = motion_lbs[avg_motion > 0]
        avg_motion = avg_motion[avg_motion > 0]
        assert len(avg_motion) == len(motion_lbs)
        
        X = avg_motion
        y = motion_lbs
        w = get_class_weight({0 : len(y[y==0]), 1 : len(y[y==1])})
    elif experiment_type == 2:
        nb_epoch=100
        y = dataset['labels']
        for v in sliding_window(y,wsize,wsize//5):
            V.append([[1,0] if e == 0 else [0,1] for e in v])
        U_players = [[] for i in range(N)] 
        
        for i in arange(1,N+1): 
            X =  array(dataset['vid_features'][i])
            
            for u in zip(sliding_window(X,wsize,wsize//5)):
                U_players[i-1].append(u)
            print(array(U_players[i-1]).shape,len(V))
        U = array([U_players[np.random.randint(0,3)][k][0] for k in range(len(V))])
        V = array(V)
    elif experiment_type == 3:
        nb_epoch=100
        X,y = array(dataset['vid_features'][0]),dataset['labels']
    elif experiment_type == 4:
        nb_epoch=100
        X,y = array(dataset['audio_features']),dataset['labels']
    if experiment_type != 2:
        for u,v in zip(sliding_window(X,wsize,wsize//5),
                        sliding_window(y,wsize,wsize//5)):
            U.append(u)
            V.append([[1,0] if e == 0 else [0,1] for e in v])
        U,V = array(U),array(V)
        if len(U.shape) == 2:
            U  = U[...,np.newaxis]
    else:
        pass
        
    print("Datset shapes:",U.shape,V.shape)
    if experiment_type != 4:
        nn = nn_model_video(data_dim=U.shape[-1],timesteps = U.shape[-2])
    else:
        nn = nn_model_audio(data_dim=U.shape[-1],timesteps = U.shape[-2])
    U_train, U_test, v_train, v_test = train_test_split(U, V, test_size=0.3, random_state=7)
    
    nn.fit(U_train,v_train,epochs=nb_epoch,batch_size=128,verbose=verbose)
    v_pred_proba = nn.predict_proba(U_test)
    v_test_hat = []
    v_pred  = []
    for seq in v_test:
        v_test_hat.append(np.argmax(seq,1))
    v_test_hat = array(v_test_hat)
    for seq in v_pred_proba:
         v_pred.append(np.argmax(seq,1))
    v_pred = array(v_pred)
    scores = {
        'jaccard': [], 
        'hamming_loss' : [],
        'accuracy' : []
    }
    night_samples = v_test_hat.sum(1)!=0
    for t,f,s in zip(v_test_hat,v_pred,v_pred_proba):
        j = jaccard_similarity_score(t,f)
        h = hamming_loss(t,f)
        acc = accuracy_score(t,f)
        scores['jaccard'].append(j)
        scores['hamming_loss'].append(h)
    
    scores['jaccard'] = array(scores['jaccard'])
    scores['hamming_loss'] = array(scores['hamming_loss'])
    #alpha = array([sum(f)/sum(t) if sum(t) > 0  and sum(f) > 0 else 0  for t,f in zip(v_test_hat,v_pred) ])
    #alpha = alpha[alpha>0]
    f1 = f1_score(v_test_hat.ravel(),v_pred.ravel())
    f1_night = f1_score(v_test_hat[night_samples].ravel(),v_pred[night_samples].ravel())
    qtable  = array([
        [np.mean(scores['hamming_loss']),np.mean(scores['jaccard']),f1],
        [np.mean(scores['hamming_loss'][night_samples]),np.mean(scores['jaccard'][night_samples]),f1_night]
    ])
    cols = ['hamming_loss','jaccard','f1*']
    qtable = pd.DataFrame(qtable,columns = cols, index = ['total','night'])
    
    return (nn,qtable,v_test_hat,v_pred,v_pred_proba)

In [ ]:
nn1,df1,v_test1,v_pred1,v_probs1 = day_night_experiment(1,verbose=2)

In [ ]:
df1

In [ ]:
nn2,df2,v_test2,v_pred2,v_probs2 = day_night_experiment(2,verbose=2)

In [ ]:
df2

In [ ]:
nn3,df3,v_test3,v_pred3,v_probs3 = day_night_experiment(3,verbose=2)

In [ ]:
df3

In [ ]:
nn4,df4,v_test4,v_pred4,v_probs4 =  day_night_experiment(4,verbose=2)

In [ ]:
df4

In [ ]:
total_df = pd.concat([df1,df2,df3,df4], axis=0,names  = ['1','2','3','4'])

In [ ]:
total_df

## Combos

### OR RULE

In [ ]:
assert np.all(v_test3 == v_test4) and np.all(v_test2 == v_test3)

In [ ]:
v_pred234 = np.logical_or(v_pred2,v_pred3,v_pred4).astype(int)

In [ ]:
night_samples = v_test3.sum(1)!=0 

In [ ]:
f1_combo = f1_score(v_pred234.ravel(),v_test3.ravel())
f1_combo_night = f1_score(v_pred234[night_samples].ravel(),v_test3[night_samples].ravel())
jaccard_combo,hamming_loss_combo = [],[]

for t,f in zip(v_pred234,v_test3):
    jaccard_combo.append(jaccard_similarity_score(t,f))
    hamming_loss_combo.append(hamming_loss(t,f))
jaccard_combo_mean,hamming_loss_mean = np.mean(jaccard_combo),np.mean(hamming_loss_combo)
jaccard_combo_mean_night = np.mean(array(jaccard_combo)[night_samples])
hamming_loss_mean_night = np.mean(array(hamming_loss_combo)[night_samples])

In [ ]:
combo_or = array([[hamming_loss_mean,jaccard_combo_mean,f1_combo],
             [hamming_loss_mean_night,jaccard_combo_mean_night,f1_combo_night]])

In [ ]:
combo_or = pd.DataFrame(combo_or,
             columns='hamming_loss,jaccard,f1*'.split(','),index = 'total,night'.split(','))

In [ ]:
combo_or

## Average  probs rule

In [ ]:
avg_proba,v_pred_avg = [],[]

In [ ]:
avg_proba = array([((p2+p3+p4)/3) for p2,p3,p4 in zip(v_probs2,v_probs3,v_probs4)])

In [ ]:
v_pred_avg = array([np.argmax(seq,1) for seq in avg_proba])

In [ ]:
f1_avg = f1_score(v_pred_avg.ravel(),v_test3.ravel())
f1_avg_night = f1_score(v_pred_avg[night_samples].ravel(),v_test3[night_samples].ravel())
jaccard_avg,hamming_loss_avg = [],[]

for t,f in zip(v_pred_avg,v_test3):
    jaccard_avg.append(jaccard_similarity_score(t,f))
    hamming_loss_avg.append(hamming_loss(t,f))
jaccard_avg_mean,hamming_loss_mean = np.mean(jaccard_avg),np.mean(hamming_loss_avg)
jaccard_avg_mean_night = np.mean(array(jaccard_avg)[night_samples])
hamming_loss_mean_night = np.mean(array(hamming_loss_avg)[night_samples])

In [ ]:
combo_avg = array([[hamming_loss_mean,jaccard_avg_mean,f1_combo],
             [hamming_loss_mean_night,jaccard_avg_mean_night,f1_avg_night]])

In [ ]:
combo_avg = pd.DataFrame(combo_avg,
             columns='hamming_loss,jaccard,f1*'.split(','),index = 'total,night'.split(','))

In [ ]:
combo_avg

In [ ]:
total_df = pd.concat([total_df,combo_or,combo_avg])

In [ ]:
total_df

In [ ]:
assert

## Saving  data and results

In [ ]:
do_save = True

In [ ]:
now = datetime.datetime.now().strftime("%Y-%m-%d__%H-%M-%S")
total_df.to_csv('total_df_combo_all_'+now+'.csv')

In [ ]:
if do_save:
    now = datetime.datetime.now().strftime("%Y-%m-%d__%H-%M-%S")
    fname = 'dataset_'+now + '.pkl'
    print(fname)
    with open(fname, 'wb') as f:
        pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)

## Werewolf detection


In [ ]:
def create_wlf_dataset(dstype = 1):
    y = []
    X = []
    for e in vlist[:]:
        vname = e.split('\\')[-1]
        aname = vname.replace('mp4','opus')
        #print(vname,aname)

        speaking_minutes = xls.parse(vname[:-4])
        samples = ['Player'+str(e) for e in arange(1,11)]
        interv = []
        mafia_list = ['Player' + str(e) for e in gtruth[gtruth.File == vname][['M1','M2','M3']].values.ravel()]
        #print(mafia_list)
        for e in samples:
            interv.append( [t2sec(e)  for e in speaking_minutes[e].values if str(e)!='nan'])
        interv = array(interv)

        audio_features = array(dataset[aname])
        for idx,e in enumerate(samples):
            video_features = array(dataset[vname][idx+1])
            e1 = interv[idx]

            for i in range(0,len(e1),2):
                A = audio_features[e1[i]:e1[i+1]]
                V = video_features[e1[i]:e1[i+1]]
                for a,v in zip(A,V) :
                    if dstype == 1:
                        X.append(np.hstack([a]))
                    elif dstype == 2:
                        X.append(np.hstack([v]))
                    elif dstype == 3:
                        X.append(np.hstack([v,a]))
                    if 'Player'+str(idx+1) in mafia_list:
                        y.append(1)
                    else:
                        y.append(0)
    X = array(X)
    y = array(y)
    return [X,y]

In [ ]:
def werewolf_experiment(X, y, resample = 1):

    params = {
            'learning_rate': 0.1, 
            'n_estimators': 1000, 
            'seed': 0, 
            'subsample': 0.8, 
            'colsample_bytree': 0.8, 
            'objective': 'binary:logistic',
    }
    if resample == 2:
        X = X.reshape([n//2,2*m])
        y = array([(y[i] or y[i+1]) for i in range(0,len(y)-1,2)])

    assert len(X) == len(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    gbt = XGBClassifier(**params)
    gbt.fit(X_train,y_train)
    y_pred = gbt.predict(X_test)
    acc,f1 = accuracy_score(y_test,y_pred),f1_score(y_test,y_pred)
    df = pd.DataFrame(confusion_matrix(y_test,y_pred))
    return [acc,f1,df]

In [ ]:
X1,y1 = create_wlf_dataset(1)

In [ ]:
X2,y2 = create_wlf_dataset(2)

In [ ]:
X3,y3 = create_wlf_dataset(3)

In [ ]:
res1 = werewolf_experiment(X=X1,y=y1)

In [ ]:
res2 = werewolf_experiment(X=X2,y=y2)

In [ ]:
res3 = werewolf_experiment(X=X3,y=y3)

In [ ]:
baseline = (accuracy_score(y1,zeros_like(y1)),
            f1_score(y1,zeros_like(y1))
                    )

In [ ]:
qtable = pd.DataFrame([baseline,res1[:2],res2[:2],res3[:2]],columns='Acc,F1'.split(','),
                   index=['Baseline','Xgb+A','Xgb+V','Xgb+A+V']
     )

In [ ]:
qtable

In [ ]:
cmatrix = pd.concat([res1[-1],res2[-1],res3[-1]])
cmatrix.index = ('Mafia(WLF)','Citizen')*3
cmatrix.columns = ('Mafia(WLF)','Citizen')

In [ ]:
cmatrix